In [1]:
import sys
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from hrvanalysis import get_time_domain_features, get_frequency_domain_features
from hrvanalysis.preprocessing import get_nn_intervals
import biosppy.signals.ecg as ecg
from tqdm import tqdm
import neurokit2 as nk
from sklearn.utils import class_weight
import json
import pyhrv.tools as tools
from pyhrv.hrv import hrv

sys.path.append('..')
from util import load_base_data

from pdb import set_trace

import warnings
warnings.simplefilter("ignore")

plt.rcParams['figure.figsize'] = [20, 5]

In [2]:
SAMPLING_RATE = 300
X_train, y_train, X_test = load_base_data(just_train=False, standard=True, flip=True)


In [ ]:
DATA_PATH = "../data/base"
SAMPLING_RATE = 300
X_test = pd.read_csv(f'{DATA_PATH}/X_test.csv')
X_train = pd.read_csv(f'{DATA_PATH}/X_train.csv')
y_train = pd.read_csv(f'{DATA_PATH}/y_train.csv', index_col='id')

In [ ]:
def read_data(dataname):
    X_test = pd.read_csv(f'../data/{dataname}/X_test.csv')
    X_train = pd.read_csv(f'../data/{dataname}/X_train.csv')
    return X_train, X_test
def concat_data(x1, x2):
    concated_x = pd.concat([x1, x2.drop(columns=['id'])], axis=1)
    return concated_x

In [3]:
def generate_interval_features(data, file_path, start=0, generate_fn=True):
    feature_names = ['nni_counter', 'nni_mean', 'nni_min', 'nni_max', 'hr_mean', 'hr_min', 'hr_max', 'hr_std', 
           'nni_diff_mean', 'nni_diff_min', 'nni_diff_max', 'sdnn', 'rmssd', 
           'sdsd', 'nn50', 'pnn50', 'nn20', 'pnn20', 'sd1', 'sd2', 'sd_ratio'] #, 'ellipse_area', 'sampen']
#     feature_vecs = []
    error_counts = 0
    
    if generate_fn:
        with open(file_path, "w") as f:
            f.write("id,"+",".join(feature_names) + "\n")
        
    for i in tqdm(range(start, len(data))):
        signal = data.loc[i].dropna().to_numpy(dtype='float32')
        feature_vector = None

        try:
            signals, rpeaks = ecg.ecg(signal, show=False)[1:3]
#             signals, info = nk.ecg_process(signal, sampling_rate=SAMPLING_RATE)
#             r_peaks = info["ECG_R_Peaks"]
            nni = tools.nn_intervals(rpeaks)
            extracted = hrv(nni, rpeaks, signals, SAMPLING_RATE)
            feature_vector = np.array([extracted[k] for k in feature_names])
        except:
            pass
        
        if feature_vector is None:
            error_counts += 1
            feature_vector = [np.nan]*len(feature_names)
#         feature_vecs.append(feature_vector)

        with open(file_path, "a") as f:
            f.write(f"{str(i)},"+",".join([str(x) for x in feature_vector])+"\n")

#     return feature_names, np.array(feature_vecs), error_counts
    return feature_names, error_counts


In [ ]:
# %%capture
feature_names, error_counts = generate_interval_features(X_train, "X_train.csv", start=2274, generate_fn=False)

 74%|████████████████████████████████████████████████████                  | 2113/2843 [1:12:42<11:09,  1.09it/s]

In [ ]:
# %%capture
feature_names, error_counts = generate_interval_features(X_test, "X_test.csv")